# Hugging Face Transformers 微调语言模型-问答任务

**注意：微调后的模型仍然是通过提取上下文的子串来回答问题的，而不是生成新的文本。**

### 模型执行问答效果示例

![Widget inference representing the QA task](docs/images/question_answering.png)

## Homework：加载本地保存的模型，进行评估和再训练更高的 F1 Score

### 关键参数

In [3]:
# 根据你使用的模型和GPU资源情况，调整以下关键参数
squad_v2 = True
model_checkpoint = "distilbert-base-uncased"
batch_size = 64

from_model_dir = f"models/s_{model_checkpoint}-finetuned-squad"
model_dir = f"models/{model_checkpoint}-finetuned-squad"

# The maximum length of a feature (question and context)
max_length = 384 
# The authorized overlap between two part of the context when splitting it is needed.
doc_stride = 128 

### 下载数据集

In [2]:
from datasets import load_dataset
datasets = load_dataset("squad_v2" if squad_v2 else "squad")
# The datasets object itself is DatasetDict, which contains one key for the training, validation and test set.
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

### 预处理数据

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

# 填充的策略
pad_on_right = tokenizer.padding_side == "right"

/root/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
def prepare_train_features(examples):
    # 一些问题的左侧可能有很多空白字符，这对我们没有用，而且会导致上下文的截断失败
    # （标记化的问题将占用大量空间）。因此，我们删除左侧的空白字符。
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # 使用截断和填充对我们的示例进行标记化，但保留溢出部分，使用步幅（stride）。
    # 当上下文很长时，这会导致一个示例可能提供多个特征，其中每个特征的上下文都与前一个特征的上下文有一些重叠。
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # 由于一个示例可能给我们提供多个特征（如果它具有很长的上下文），我们需要一个从特征到其对应示例的映射。这个键就提供了这个映射关系。
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # 偏移映射将为我们提供从令牌到原始上下文中的字符位置的映射。这将帮助我们计算开始位置和结束位置。
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # 让我们为这些示例进行标记！
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # 我们将使用 CLS 特殊 token 的索引来标记不可能的答案。
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # 获取与该示例对应的序列（以了解上下文和问题是什么）。
        sequence_ids = tokenized_examples.sequence_ids(i)

        # 一个示例可以提供多个跨度，这是包含此文本跨度的示例的索引。
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # 如果没有给出答案，则将cls_index设置为答案。
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # 答案在文本中的开始和结束字符索引。
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # 当前跨度在文本中的开始令牌索引。
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # 当前跨度在文本中的结束令牌索引。
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # 检测答案是否超出跨度（在这种情况下，该特征的标签将使用CLS索引）。
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # 否则，将token_start_index和token_end_index移到答案的两端。
                # 注意：如果答案是最后一个单词（边缘情况），我们可以在最后一个偏移之后继续。
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [6]:
tokenized_datasets = datasets.map(prepare_train_features,
                                  batched=True,
                                  remove_columns=datasets["train"].column_names)

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

### 微调模型

In [7]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
trained_model = AutoModelForQuestionAnswering.from_pretrained(from_model_dir)

In [8]:
# 训练超参数（TrainingArguments）
args = TrainingArguments(
    output_dir=model_dir,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=5
)

In [10]:
# Data Collator（数据整理器）
# 数据整理器将训练数据整理为批次数据，用于模型训练时的批次处理。本教程使用默认的 default_data_collator。
from transformers import default_data_collator
data_collator = default_data_collator

In [11]:
trained_trainer = Trainer(
    trained_model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [13]:
trained_trainer.train()

Epoch,Training Loss,Validation Loss
1,1.092000,1.235397
2,0.889600,1.216630
3,0.768800,1.358643


TrainOutput(global_step=6177, training_loss=0.9445470062991996, metrics={'train_runtime': 12676.3889, 'train_samples_per_second': 31.181, 'train_steps_per_second': 0.487, 'total_flos': 3.873165421863629e+16, 'train_loss': 0.9445470062991996, 'epoch': 3.0})

In [ ]:
# trained_trainer.train(resume_from_checkpoint=True)

In [15]:
# 训练完成后，第一时间保存模型权重文件。
model_to_save = trained_trainer.save_model(model_dir)
trained_trainer.save_state()

### 模型评估

In [16]:
n_best_size = 20
max_answer_length = 30

#### 预处理验证集

In [17]:
def prepare_validation_features(examples):
    # 一些问题的左侧有很多空白，这些空白并不有用且会导致上下文截断失败（分词后的问题会占用很多空间）。
    # 因此我们移除这些左侧空白
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # 使用截断和可能的填充对我们的示例进行分词，但使用步长保留溢出的令牌。这导致一个长上下文的示例可能产生
    # 几个特征，每个特征的上下文都会稍微与前一个特征的上下文重叠。
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # 由于一个示例在上下文很长时可能会产生几个特征，我们需要一个从特征映射到其对应示例的映射。这个键就是为了这个目的。
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # 我们保留产生这个特征的示例ID，并且会存储偏移映射。
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # 获取与该示例对应的序列（以了解哪些是上下文，哪些是问题）。
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # 一个示例可以产生几个文本段，这里是包含该文本段的示例的索引。
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # 将不属于context的offset_mapping设置为None，以便容易确定一个token位置是否属于上context。
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [18]:
# 将prepare_validation_features应用到整个验证集：
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

#### Predict

In [20]:
raw_predictions = trained_trainer.predict(validation_features)
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

#### 后处理预测值

In [23]:
from tqdm.auto import tqdm
import collections
import numpy as np

def map_features_to_examples(examples, features):
    # 创建从示例 ID 到其索引的映射
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}

    # 创建一个默认字典，用于存储每个示例对应的特征索引列表
    features_per_example = collections.defaultdict(list)

    # 遍历所有特征，将特征索引添加到对应的示例中
    for i, feature in enumerate(features):
        example_index = example_id_to_index[feature["example_id"]]
        features_per_example[example_index].append(i)

    return features_per_example

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # 构建一个从示例到其对应特征的映射。
    features_per_example = map_features_to_examples(examples, features)

    # 我们需要填充的字典。
    predictions = collections.OrderedDict()

    # 日志记录。
    print(f"正在后处理 {len(examples)} 个示例的预测，这些预测分散在 {len(features)} 个特征中。")

    # 遍历所有示例！
    for example_index, example in enumerate(tqdm(examples)):
        # 这些是与当前示例关联的特征的索引。
        feature_indices = features_per_example[example_index]

        min_null_score = None # 仅在squad_v2为True时使用。
        valid_answers = []
        
        context = example["context"]
        # 遍历与当前示例关联的所有特征。
        for feature_index in feature_indices:
            # 我们获取模型对这个特征的预测。
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # 这将允许我们将logits中的某些位置映射到原始上下文中的文本跨度。
            offset_mapping = features[feature_index]["offset_mapping"]

            # 更新最小空预测。
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # 浏览所有的最佳开始和结束logits，为 `n_best_size` 个最佳选择。
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # 不考虑超出范围的答案，原因是索引超出范围或对应于输入ID的部分不在上下文中。
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # 不考虑长度小于0或大于max_answer_length的答案。
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # 在极少数情况下我们没有一个非空预测，我们创建一个假预测以避免失败。
            best_answer = {"text": "", "score": 0.0}
        
        # 选择我们的最终答案：最佳答案或空答案（仅适用于squad_v2）
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [24]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

正在后处理 11873 个示例的预测，这些预测分散在 12134 个特征中。


  0%|          | 0/11873 [00:00<?, ?it/s]

#### 加载评估指标

In [4]:
from datasets import load_metric

metric = load_metric("squad_v2" if squad_v2 else "squad")

/tmp/ipykernel_3193443/2330875496.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2" if squad_v2 else "squad")
/root/miniconda3/envs/peft/lib/python3.10/site-packages/datasets/load.py:752: FutureWarning: The repository for squad_v2 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/squad_v2/squad_v2.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


#### 计算评估值

In [26]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact': 62.73056514781437,
 'f1': 66.35280746076715,
 'total': 11873,
 'HasAns_exact': 66.75101214574899,
 'HasAns_f1': 74.0058844436042,
 'HasAns_total': 5928,
 'NoAns_exact': 58.72161480235492,
 'NoAns_f1': 58.72161480235492,
 'NoAns_total': 5945,
 'best_exact': 62.73056514781437,
 'best_exact_thresh': 0.0,
 'best_f1': 66.35280746076728,
 'best_f1_thresh': 0.0}

第三个epoch后，Validation Loss突然上升，最终评估的F1值下降了。

灾难性遗忘（Catastrophic Forgetting）是深度学习中一个常见的问题，尤其是在顺序学习（sequential learning）或连续学习（continual learning）中表现得尤为明显。它指的是模型在学习新任务时，会快速遗忘之前学过的任务。这在处理多任务学习或在线学习时尤其需要注意。以下是对灾难性遗忘的详细解释以及一些常见的解决方案：

### 灾难性遗忘的解释

当模型在没有特定策略的情况下按顺序学习不同的任务时，模型的参数会朝着优化新任务的方向更新，可能会破坏之前任务所需的参数设置。结果是模型在新任务上的表现可能会很好，但在先前任务上的表现会显著下降。

### 解决灾难性遗忘的方法

1. **正则化方法**
   - **Elastic Weight Consolidation (EWC)**: 在训练新任务时，通过增加一个正则项来惩罚与之前任务重要参数的偏离，从而保留以前任务的重要知识。
   - **Synaptic Intelligence (SI)**: 类似于 EWC，但它通过追踪参数在不同任务中的重要性并在损失函数中添加正则项来保留重要参数。

2. **回放方法**
   - **经验回放（Experience Replay）**: 将先前任务的样本存储起来，在训练新任务时与新任务样本一起训练模型。这可以是直接回放样本（如贪婪回放）或生成对抗网络生成的样本。
   - **渐进神经网络（Progressive Neural Networks）**: 为每个新任务增加新的专用神经网络模块，原始模块的参数冻结，不再更新，但通过侧链连接共享信息。

3. **参数隔离方法**
   - **固定子网络（Fixed Subnetworks）**: 为每个任务分配不同的参数子集，并确保这些子集在后续任务中不会被修改。
   - **掩码网络（Masked Networks）**: 使用掩码来控制不同任务激活不同的参数子集，从而减少不同任务间的参数冲突。

4. **模型扩展方法**
   - **动态扩展神经网络（Dynamic Expandable Networks）**: 根据新任务的需要动态增加网络结构和参数，以适应新的任务。

### 代码示例：使用 EWC 解决灾难性遗忘

以下是使用 PyTorch 实现 EWC 方法的简化示例：

```python
import torch
import torch.nn as nn
import torch.optim as optim

class EWC:
    def __init__(self, model, dataloader, lambda_ewc):
        self.model = model
        self.dataloader = dataloader
        self.lambda_ewc = lambda_ewc
        self.params = {n: p for n, p in model.named_parameters() if p.requires_grad}
        self._means = {}
        self._precision_matrices = self._diag_fisher()
        for n, p in self.params.items():
            self._means[n] = p.clone().detach()

    def _diag_fisher(self):
        precision_matrices = {n: torch.zeros_like(p) for n, p in self.params.items()}
        self.model.eval()
        for input, target in self.dataloader:
            self.model.zero_grad()
            output = self.model(input)
            loss = nn.CrossEntropyLoss()(output, target)
            loss.backward()
            for n, p in self.params.items():
                precision_matrices[n].data += p.grad.data ** 2 / len(self.dataloader)
        precision_matrices = {n: p for n, p in precision_matrices.items()}
        return precision_matrices

    def penalty(self):
        loss = 0
        for n, p in self.model.named_parameters():
            if n in self._means:
                _loss = self._precision_matrices[n] * (p - self._means[n]) ** 2
                loss += _loss.sum()
        return loss

    def __call__(self, model_output, target):
        loss = nn.CrossEntropyLoss()(model_output, target)
        ewc_loss = self.penalty()
        return loss + self.lambda_ewc * ewc_loss

# 假设我们有一个预训练的 model 和 dataloader
model = ...  # Your model
dataloader = ...  # Your dataloader for the old task
lambda_ewc = 0.4  # Regularization strength

# 初始化 EWC 实例
ewc = EWC(model, dataloader, lambda_ewc)

# 在新任务上训练
optimizer = optim.Adam(model.parameters(), lr=1e-4)
for input, target in new_task_dataloader:
    optimizer.zero_grad()
    output = model(input)
    loss = ewc(output, target)
    loss.backward()
    optimizer.step()
```

### 结论

灾难性遗忘是连续学习中的一个重要问题，但通过使用正则化方法、回放方法、参数隔离方法以及模型扩展方法，可以有效地缓解这一问题。选择合适的方法取决于具体的应用场景和计算资源限制。

**这是在squad数据集的训练结果上，使用squad v2再训练的，如果两次都用squad v2会不会好点？homework4再试试**